# Rule-Filtered Vector Similarity Retrieval

This notebook demonstrates a two-stage retrieval approach:
1. **Filter by rule**: Match the query rule to a rule category
2. **Retrieve top K**: Get most similar examples from that rule's examples using vector similarity

## 1. Setup and Imports

In [4]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from utils.rule_filtered_retrieval import FastEmbedder, RuleFilteredRetriever
from sklearn.metrics import f1_score, classification_report

## 2. Load Training Data

In [5]:
# Load training data
base_path = "./data/final/zothers/"
df_train = pd.read_csv(f"{base_path}rule_comment.csv")
df_train['value'] = df_train['value'].replace(0, -1)
df_train = df_train.sample(frac=1).reset_index(drop=True)

print(f"Total training examples: {len(df_train):,}")
print(f"Unique rules: {df_train['rule'].nunique():,}")
print(f"\nColumns: {df_train.columns.tolist()}")

# Show rule distribution
rule_counts = df_train['rule'].value_counts()
print(f"\nRule distribution statistics:")
print(f"  Mean examples per rule: {rule_counts.mean():.1f}")
print(f"  Median examples per rule: {rule_counts.median():.1f}")
print(f"  Min examples per rule: {rule_counts.min()}")
print(f"  Max examples per rule: {rule_counts.max()}")

df_train.head()
print(df_train["value"].unique())

Total training examples: 251,740
Unique rules: 6

Columns: ['rule', 'test_comment', 'violates_rule', 'value']

Rule distribution statistics:
  Mean examples per rule: 41956.7
  Median examples per rule: 45187.5
  Min examples per rule: 34585
  Max examples per rule: 46215
[-1  1]


In [6]:
# Sample for faster experimentation (optional)
SAMPLE_SIZE = 251739 # Set to None to use all data

if SAMPLE_SIZE and SAMPLE_SIZE < len(df_train):
    df_train_sample = df_train.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)
    print(f"Using {len(df_train_sample):,} samples for faster experimentation")
else:
    df_train_sample = df_train
    print(f"Using all {len(df_train_sample):,} examples")

print(f"Unique rules in sample: {df_train_sample['rule'].nunique():,}")

Using 251,739 samples for faster experimentation
Unique rules in sample: 6


## 3. Initialize Rule-Filtered Retriever

In [7]:
# Initialize embedder
print("Loading Qwen3-Embedding model...")
embedder = FastEmbedder(
    model_name='Qwen/Qwen3-Embedding-0.6B',
    output_dim=1024  # Can experiment with 512, 1024, or full dimension
)

print(f"Embedder initialized on device: {embedder.device}")

Loading Qwen3-Embedding model...
Embedder initialized on device: cuda


In [8]:
# Initialize retriever and build index
print("Initializing retriever...")
retriever = RuleFilteredRetriever(embedder, use_gpu=True)

# Define instruction for better retrieval
instruction = "Given a rule and comment, retrieve similar training examples for classification"

# Build index (this creates separate FAISS index for each rule)
print("\nBuilding rule-filtered indices...")
retriever.build_index(
    df_train_sample,
    rule_col='rule',
    comment_col='test_comment',
    value_col='value',
    instruction=instruction
)

print("\n✓ Index building complete!")

Initializing retriever...

Building rule-filtered indices...
Building rule-filtered index from 251739 examples...
Strategy: Filter by rule FIRST, then search by COMMENT similarity only
Found 6 unique rules
Encoding comments only (not combined with rules)...
Building per-rule FAISS indices...
  Rule 'No Advertising: Spam, referral links, unsolicited ...' : 34599 examples
  Rule 'No legal advice: Do not offer or request legal adv...' : 34585 examples
  Rule 'no financial advice: we do not permit comments tha...' : 46215 examples
  Rule 'no medical advice: do not offer or request specifi...' : 45965 examples
  Rule 'no promotion of illegal activity: do not encourage...' : 45245 examples
  Rule 'no spoilers: do not reveal important details that ...' : 45130 examples

Index building complete!
Total rules: 6
Total examples: 251739

✓ Index building complete!


## 4. Test Single Query Retrieval

In [9]:
# Test with a random example
test_idx = 42
test_row = df_train_sample.iloc[test_idx]

print(f"Query Rule: {test_row['rule'][:200]}...")
print(f"\nQuery Comment: {test_row['test_comment'][:200]}...")
print(f"\nTrue Value: {test_row['value']}")

# Retrieve similar examples
result = retriever.retrieve(
    query_rule=test_row['rule'],
    query_comment=test_row['test_comment'],
    top_k=10,
    instruction=instruction
)

print(f"\n{'='*80}")
print("RETRIEVAL RESULTS")
print(f"{'='*80}")
print(f"Matched Rule Type: {result['rule_match_type']}")
print(f"Number of examples in this rule category: {result['num_examples_in_rule']}")
print(f"Retrieved: {len(result['examples'])} examples")

print(f"\nTop 5 Similarities:")
for i, (sim, ex) in enumerate(zip(result['similarities'][:5], result['examples'][:5])):
    print(f"  {i+1}. Similarity: {sim:.4f}, Value: {ex['value']}, Comment: {ex['test_comment'][:80]}...")

# Calculate prediction
retrieved_values = [ex['value'] for ex in result['examples']]
predicted_value = np.mean(retrieved_values)
predicted_decision = 1 if predicted_value > 0 else 0

print(f"\nPredicted Value: {predicted_value:.3f}")
print(f"Predicted Decision: {predicted_decision}")
print(f"True Value: {test_row['value']}")

Query Rule: no promotion of illegal activity: do not encourage or promote illegal activities, such as drug-related activity, violence, exploitation, theft, or other criminal behavior....

Query Comment: "Instead of stealing lab supplies, why not check if there are any grants or funding options available? It’s a lot less risky and more ethical."...

True Value: -1

RETRIEVAL RESULTS
Matched Rule Type: exact
Number of examples in this rule category: 45245
Retrieved: 10 examples

Top 5 Similarities:
  1. Similarity: 1.0000, Value: -1, Comment: "Instead of stealing lab supplies, why not check if there are any grants or fund...
  2. Similarity: 0.7566, Value: 1, Comment: "Why not just break into a research lab and steal the equipment? It’s not like t...
  3. Similarity: 0.7383, Value: -1, Comment: "Instead of trying to get funding through illegal means, why not apply for more ...
  4. Similarity: 0.7058, Value: -1, Comment: Instead of stealing, why not look for odd jobs or community resourc

## 5. Batch Evaluation on Test Set

In [10]:
# Load test data
df_test = pd.read_csv("./data/final/df_test_cr_12.csv")

print(f"Test set size: {len(df_test):,}")
print(f"Unique rules in test: {df_test['rule'].nunique():,}")

df_test.head()

Test set size: 2,000
Unique rules in test: 6


,subreddit,rule,positive_example_1,negative_example_1,positive_example_2,negative_example_2,test_comment,violates_rule
0,personalfinance,"No Advertising: Spam, referral links, unsolici...","Hey everyone, I've been using Wealthfront for ...",I've had good luck with Vanguard index funds f...,"As a certified financial planner, I'm offering...",When my emergency fund hit $10k I moved it to ...,I work for Acme Investing and we're running a ...,Yes
1,tifu,no promotion of illegal activity: do not encou...,Just go back after hours and jimmy the window ...,My friend once accidentally set off the fire a...,Honestly dude just lace their drinks with some...,"I can't believe I told my boss ""love you"" inst...","Next time they pull that shit, just slash thei...",Yes
2,OutOfTheLoop,no promotion of illegal activity: do not encou...,"Just torrent the movie bro, studios make enoug...",Some people use VPNs to access geo-restricted ...,"If your landlord is screwing you over, just st...",There are legitimate ways to protest unfair po...,"Honestly if someone steals from you, just brea...",Yes
3,nottheonion,no medical advice: do not offer or request spe...,"For that persistent cough you mentioned, you s...",Wow this article about hospitals using leeches...,"OP if you're experiencing those symptoms, stop...",Reading this makes me want to schedule my annu...,"If your headache lasts more than 2 days, you p...",Yes
4,Christianity,no medical advice: do not offer or request spe...,Hey guys I've been having these intense migrai...,Praying for everyone struggling with health is...,My anxiety is through the roof lately and my Z...,Has anyone found comfort in specific Psalms du...,My daughter has a 102 fever that won't break w...,Yes


In [11]:
# Batch retrieval with different top_k values
TOP_K_VALUES = [10]

results_dict = {}

for top_k in TOP_K_VALUES:
    print(f"\nEvaluating with top_k={top_k}...")
    
    results_df = retriever.batch_retrieve(
        df_test,
        rule_col='rule',
        comment_col='test_comment',
        top_k=top_k,
        instruction=instruction
    )
    
    results_dict[top_k] = results_df
    
    # Calculate metrics
    y_true = df_test["violates_rule"].astype(str).str.strip().str.strip('"').str.strip("'").str.lower().map({"yes": 1, "no": 0})
    y_pred = results_df["decision"].values
    
    f1 = f1_score(y_true, y_pred)
    print(f"  F1-Score: {f1:.4f}")
    
    print(f"  Rule match types:")
    print(results_df['rule_match_type'].value_counts().to_string())

print("\n✓ Batch evaluation complete!")


Evaluating with top_k=10...
  F1-Score: 0.9861
  Rule match types:
rule_match_type
exact    2000

✓ Batch evaluation complete!


## 6. Detailed Analysis

In [12]:
# Choose best performing top_k
best_k = 10
results_df = results_dict[best_k]

# Prepare ground truth
y_true = df_test["violates_rule"].astype(str).str.strip().str.strip('"').str.strip("'").str.lower().map({"yes": 1, "no": 0})
y_pred = results_df["decision"].values

# Classification report
print(f"Classification Report (top_k={best_k}):")
print("="*80)
print(classification_report(y_true, y_pred, target_names=['No Violation', 'Violation']))

# Text-based confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print("="*80)
print(f"                    Predicted No    Predicted Yes")
print(f"Actual No           {cm[0,0]:>12}    {cm[0,1]:>13}")
print(f"Actual Yes          {cm[1,0]:>12}    {cm[1,1]:>13}")
print("="*80)

# Additional metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("\nDetailed Metrics:")
print(f"  Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
print(f"  Precision: {precision_score(y_true, y_pred):.4f}")
print(f"  Recall:    {recall_score(y_true, y_pred):.4f}")
print(f"  F1-Score:  {f1_score(y_true, y_pred):.4f}")

Classification Report (top_k=10):
              precision    recall  f1-score   support

No Violation       0.99      0.98      0.99      1000
   Violation       0.98      0.99      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000


Confusion Matrix:
                    Predicted No    Predicted Yes
Actual No                    981               19
Actual Yes                     9              991

Detailed Metrics:
  Accuracy:  0.9860
  Precision: 0.9812
  Recall:    0.9910
  F1-Score:  0.9861


In [13]:
# Compare different top_k values
comparison_data = []

for top_k, results_df in results_dict.items():
    y_true = df_test["violates_rule"].astype(str).str.strip().str.strip('"').str.strip("'").str.lower().map({"yes": 1, "no": 0})
    y_pred = results_df["decision"].values
    
    f1 = f1_score(y_true, y_pred)
    
    from sklearn.metrics import precision_score, recall_score, accuracy_score
    
    comparison_data.append({
        'top_k': top_k,
        'f1_score': f1,
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'accuracy': accuracy_score(y_true, y_pred)
    })

comparison_df = pd.DataFrame(comparison_data)
print("Performance Comparison Across Different Top-K Values:")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Show best performing top_k
best_row = comparison_df.loc[comparison_df['f1_score'].idxmax()]
print(f"\nBest performing configuration:")
print(f"  Top-K: {int(best_row['top_k'])}")
print(f"  F1-Score: {best_row['f1_score']:.4f}")
print(f"  Precision: {best_row['precision']:.4f}")
print(f"  Recall: {best_row['recall']:.4f}")
print(f"  Accuracy: {best_row['accuracy']:.4f}")

Performance Comparison Across Different Top-K Values:
 top_k  f1_score  precision  recall  accuracy
    10   0.98607   0.981188   0.991     0.986

Best performing configuration:
  Top-K: 10
  F1-Score: 0.9861
  Precision: 0.9812
  Recall: 0.9910
  Accuracy: 0.9860


## 7. Save/Load Index for Future Use

In [14]:
# ============================================================================
# OPTION 1: Save the index for later use
# ============================================================================

# Save the built index
save_dir = "./saved_indices/rule_filtered_sample"
print(f"Saving index to {save_dir}...")
retriever.save_index(save_dir)

print("\n✓ Index saved! You can now load it later without rebuilding.")
print(f"  Files saved in: {save_dir}")
print("\nTo use it later:")
print("  1. Initialize embedder and retriever")
print("  2. Call: retriever.load_index(save_dir)")
print("  3. Use retriever.retrieve() or retriever.batch_retrieve()")


Saving index to ./saved_indices/rule_filtered_sample...
Saving index to ./saved_indices/rule_filtered_sample...
✓ Index saved successfully!
  - 6 FAISS indices
  - 251739 training examples
  - Embeddings shape: (251739, 1024)

✓ Index saved! You can now load it later without rebuilding.
  Files saved in: ./saved_indices/rule_filtered_sample

To use it later:
  1. Initialize embedder and retriever
  2. Call: retriever.load_index(save_dir)
  3. Use retriever.retrieve() or retriever.batch_retrieve()
